# Grabbing Jewel's SnowPit Data

1. Grab pit and format correctly to align with SnowEx database
2. Loop through folder of .csv's and create correct temperature profiles
3. Merge all profiles into one "dictionary"??? geopandas magic dataframe?

GoogleDrive PitDataTemperatures Folder: https://drive.google.com/drive/folders/1SFyBKULqiRLi52yiKO5mxRn5Icsp83Lm?usp=sharing

Test Blob (single temp profile .csv): 1DufHRYtWqxQUFrfqdFx1FViFbaTCkmAV

In [1]:
# set up and select a pit to work with
import pandas as pd

# import necessary libraries
import pandas as pd
import os
import glob
import geopandas   
# add lat.lon data
import utm  
# use glob to get all the csv files 
# in the folder
path = '../Zachary_Miller/pits/'
csv_files = glob.glob(os.path.join(path, "*.csv"))
complete_df = pd.DataFrame()
layer_df = pd.DataFrame()

def utm_to_latlon(coords, zone_number, zone_letter):
    easting = coords[0]
    northing = coords[1]
    return utm.to_latlon(easting, northing, zone_number, zone_letter)


  
#loop over the list of csv files
for f in csv_files:
    print(f)
    df = pd.read_csv(f, header=None)
    dfdata = pd.read_csv(f, header = 13)
    # grab the metadata only
    dfmeta = df.head(12)
    # transpose the metadata to columns
    dftranspose = dfmeta.transpose().head(2)
    # grab column names from the first row
    newheader = dftranspose.iloc[0]
    # remove the column names row
    dftranspose = dftranspose[1:]

    # add column names to transposed metadata
    dftranspose.columns = newheader

    # duplicate rows of metadata to length of snow pit temperature profile
    # duplicate rows of metadata to length of snow pit temperature profile
    dfdup = dftranspose.reindex(dftranspose.index.repeat(len(dfdata))).reset_index(drop=True)
    
    # merge dfdata with dfdup to make a complete df of snow profile temps
    dfmerged = pd.concat([dfdup, dfdata], axis = 1)
    del dfmerged['Hs:'] # remove the total HS measurement column

    # create snow pit location geometry
    gdf = geopandas.GeoDataFrame(
    dfmerged, geometry=geopandas.points_from_xy(dfmerged['Easting:'], dfmerged['Northing:']))

    #complete_df = complete_df.append(Sitedf)
    layer_df = layer_df.append(dfmerged)
    # LayerData dataframe
    #print(dfreorder.head)


../Zachary_Miller/pits/cl_o_am_031820_0925.csv
../Zachary_Miller/pits/sw_o_pm_012220_1455.csv
../Zachary_Miller/pits/mw_t_pm_040920_1540.csv
../Zachary_Miller/pits/mw_o_pm_040920_1318.csv
../Zachary_Miller/pits/mw_o_pm_040920_1715.csv
../Zachary_Miller/pits/mw_t_pm_031620_1328.csv
../Zachary_Miller/pits/cl_o_am_031220_0958.csv
../Zachary_Miller/pits/mw_o_pm_031620_1458.csv
../Zachary_Miller/pits/cl_o_am_032520_1104.csv
../Zachary_Miller/pits/mw_o_am_031620_0941.csv
../Zachary_Miller/pits/sw_t_am_040120_1000.csv
../Zachary_Miller/pits/mw_o_am_033120_1034.csv
../Zachary_Miller/pits/mw_t_am_040920_0825.csv
../Zachary_Miller/pits/mw_t_pm_033120_1245.csv
../Zachary_Miller/pits/sw_t_pm_032820_1519.csv
../Zachary_Miller/pits/sw_o_pm_041920_1700.csv
../Zachary_Miller/pits/sw_o_pm_042220_1612.csv
../Zachary_Miller/pits/mw_o_pm_033120_1630.csv
../Zachary_Miller/pits/cl_t_pm_032520_1322.csv
../Zachary_Miller/pits/sw_o_pm_032820_1714.csv
../Zachary_Miller/pits/sw_t_pm_031220_1449.csv
../Zachary_Mi

Pandas subsetting during imports: we'll read in the snow temp data and meta data separately then transpose, duplicate and combine the data

In [2]:
layer_df.shape

(641, 17)

In [3]:
layer_df

,Location:,Type:,Easting:,Northing:,Surveyor:,Time:,Time Type:,Air Temp:,Ground:,Notes:,Wx:,Hs,Temperature,Notes,geometry,lat_lon_tuple,Wx:
0,County Line,Open,756929,4324348,J. Lund,3/18/20 9:25,AM,-5.1,"Rough, bare.",Temperature 9:25-9:35.,"Overcast, moderate wind, very light snow.",120,-5.3,NaN,POINT (756929.000 4324348.000),"[(39.03044494848296, -102.03187482502331)]",NaN
1,County Line,Open,756929,4324348,J. Lund,3/18/20 9:25,AM,-5.1,"Rough, bare.",Temperature 9:25-9:35.,"Overcast, moderate wind, very light snow.",110,-6.0,NaN,POINT (756929.000 4324348.000),"[(39.03044494848296, -102.03187482502331)]",NaN
2,County Line,Open,756929,4324348,J. Lund,3/18/20 9:25,AM,-5.1,"Rough, bare.",Temperature 9:25-9:35.,"Overcast, moderate wind, very light snow.",100,-5.2,NaN,POINT (756929.000 4324348.000),"[(39.03044494848296, -102.03187482502331)]",NaN
3,County Line,Open,756929,4324348,J. Lund,3/18/20 9:25,AM,-5.1,"Rough, bare.",Temperature 9:25-9:35.,"Overcast, moderate wind, very light snow.",90,-4.3,NaN,POINT (756929.000 4324348.000),"[(39.03044494848296, -102.03187482502331)]",NaN
4,County Line,Open,756929,4324348,J. Lund,3/18/20 9:25,AM,-5.1,"Rough, bare.",Temperature 9:25-9:35.,"Overcast, moderate wind, very light snow.",80,-3.6,NaN,POINT (756929.000 4324348.000),"[(39.03044494848296, -102.03187482502331)]",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,Mesa West,Open,741161,4324223,J. Lund,4/9/20 6:50,AM,-2.6,"Smooth, grass/shrubs ~15cm","Water horizons @ 114, 110, 108, 105-95. Col @ ...","Clear, cold, calm/light breeze.",40,-0.8,NaN,POINT (741161.000 4324223.000),"[(39.03381144332627, -102.21386853702215)]",NaN
11,Mesa West,Open,741161,4324223,J. Lund,4/9/20 6:50,AM,-2.6,"Smooth, grass/shrubs ~15cm","Water horizons @ 114, 110, 108, 105-95. Col @ ...","Clear, cold, calm/light breeze.",30,-0.7,NaN,POINT (741161.000 4324223.000),"[(39.03381144332627, -102.21386853702215)]",NaN
12,Mesa West,Open,741161,4324223,J. Lund,4/9/20 6:50,AM,-2.6,"Smooth, grass/shrubs ~15cm","Water horizons @ 114, 110, 108, 105-95. Col @ ...","Clear, cold, calm/light breeze.",20,-0.6,NaN,POINT (741161.000 4324223.000),"[(39.03381144332627, -102.21386853702215)]",NaN
13,Mesa West,Open,741161,4324223,J. Lund,4/9/20 6:50,AM,-2.6,"Smooth, grass/shrubs ~15cm","Water horizons @ 114, 110, 108, 105-95. Col @ ...","Clear, cold, calm/light breeze.",10,-0.4,NaN,POINT (741161.000 4324223.000),"[(39.03381144332627, -102.21386853702215)]",NaN


In [15]:
import numpy as np
import hvplot.pandas  # noqa
import hvplot.dask  # noqa

In [16]:
layer_df.hvplot.line(y='Temperature', x='Time:', groupby=['Location:','Hs'], height=500)

:DynamicMap   [Location:,Hs]
   :Curve   [Time:]   (Temperature)

In [17]:
layer_df.hvplot.line(y='Hs', x='Temperature', groupby=['Location:','Time:'], height=500)

:DynamicMap   [Location:,Time:]
   :Curve   [Temperature]   (Hs)